<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
page=requests.get(static_url).text # use requests.get() method with the provided static_url
# assign the response to a object

Create a `BeautifulSoup` object from the HTML `response`


In [8]:
soup=BeautifulSoup(page,"html.parser")# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly 


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [64]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all("tr")
html_tables[3]

<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>4 June 2010,<br/>18:45
</td>
<td><a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">F9 v1.0</a><sup class="reference" id="cite_ref-MuskMay2012_13-0"><a href="#cite_note-MuskMay2012-13">[7]</a></sup><br/>B0003.1<sup class="reference" id="cite_ref-block_numbers_14-0"><a href="#cite_note-block_numbers-14">[8]</a></sup>
</td>
<td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">CCAFS</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">SLC-40</a>
</td>
<td><a href="/wiki/Dragon_Spacecraft_Qualification_Unit" title="Dragon Spacecraft Qualification Unit">Dragon Spacecraft Qualification Unit</a>
</td>
<td>
</td>
<td><a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">LEO</a>
</td>
<td><a href="/wiki/SpaceX" title="SpaceX">SpaceX</a>
</td>
<td class="table-success" style="background: #9EFF9E; vertical-align: middle; 

Starting from the third table is our target table contains the actual launch records.


In [14]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [26]:
column_names = []
for a in first_launch_table:
    column_names.append(a)
    

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [28]:
column_names.pop(0)

'\n'

In [46]:
column_names = []
for a in first_launch_table:
    b=a.text
    column_names.append(b)

In [49]:
new_column_names=[]
for a in column_names:
    b=a.replace("\n","")
    new_column_names.append(b)

In [53]:
new_column_names.remove("")

In [55]:
while new_column_names.count("")>0:
    new_column_names.remove("")
    

In [57]:
column_names=new_column_names

In [92]:
column_names

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster [b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [95]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date andtime (UTC)']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload[c]'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launchoutcome'] = []
# Added some new columns
launch_dict['Version,Booster[b]']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [101]:
launch_dict.pop("Version,Booster [b]")

In [102]:
launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload[c]': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launchoutcome': [],
 'Boosterlanding': None,
 'Version,Booster[b]': [],
 'Date': [],
 'Time': []}

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [74]:
table_content=soup.find_all("tbody")
table_content=table_content[9]

In [89]:
table_content_int=table_content.find_all("tr")
table_content_int=table_content_int[1:]

In [111]:
table_ce=[]
for a in table_content_int:
    for b in a:
        c=b.text
        table_ce.append(c)
        

In [117]:
table_ceb=[]
for n in table_ce:
    a=n.replace("\n","",)
    table_ceb.append(a)

    

In [118]:
table_ce=table_ceb

In [120]:
while table_ce.count("")>0:
    table_ce.remove("")

In [124]:
launch_dict['Flight No.']=table_ce[::11]

In [127]:
launch_dict['Version,Booster[b]']=table_ce[2::11]

In [129]:
launch_dict['Launch site'] =table_ce[3::11]

In [131]:
launch_dict['Payload[c]']=table_ce[4::11]

In [201]:
xasil=[]
for i in table_ce[5::11]:
    a=i.replace("\xa0kg","")
    xasil.append(a)
xasil

['15,600 (34,400\xa0lb)[5]',
 '12,050 (26,570\xa0lb)',
 '15,600 (34,400\xa0lb)[5]',
 '15,600 (34,400\xa0lb)[5]',
 '1,977 (4,359\xa0lb)[507]',
 '15,600 (34,400\xa0lb)[5]',
 '15,600 (34,400\xa0lb)[5]',
 '12,530 (27,620\xa0lb)[519]',
 '15,600 (34,400\xa0lb)[5]',
 '15,410 (33,970\xa0lb)[523]',
 '4,311 (9,504\xa0lb)[530]',
 '5,000–6,000 (11,000–13,000\xa0lb)',
 '14,932 (32,919\xa0lb)',
 '~15,440 (34,040\xa0lb)',
 '3,130 (6,900\xa0lb)[558]',
 '15,600 (34,400\xa0lb)[5]',
 '15,600 (34,400\xa0lb)[5]',
 '15,600 (34,400\xa0lb)[5]',
 '15,600 (34,400\xa0lb)',
 '4,311 (9,504\xa0lb)',
 '~12,500 (27,600\xa0lb)',
 '1,192 (2,628\xa0lb)',
 '15,600 (34,400\xa0lb)',
 '2,972 (6,552\xa0lb)',
 '7,000 (15,000\xa0lb)',
 'Classified']

In [202]:
xasil1=[]
for i in xasil:
    beginning=i.find("(")
    a=i[:beginning]
    xasil1.append(a)
xasil=xasil1
xasil

['15,600 ',
 '12,050 ',
 '15,600 ',
 '15,600 ',
 '1,977 ',
 '15,600 ',
 '15,600 ',
 '12,530 ',
 '15,600 ',
 '15,410 ',
 '4,311 ',
 '5,000–6,000 ',
 '14,932 ',
 '~15,440 ',
 '3,130 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '4,311 ',
 '~12,500 ',
 '1,192 ',
 '15,600 ',
 '2,972 ',
 '7,000 ',
 'Classifie']

In [203]:
a=10
b=11
if a<b:
    print(a)

10


In [210]:
xasil1=[]
for a in xasil:

    if a == "5,000–6,000 ":
        a="5,500"
    xasil1.append(a)
xasil1

['15,600 ',
 '12,050 ',
 '15,600 ',
 '15,600 ',
 '1,977 ',
 '15,600 ',
 '15,600 ',
 '12,530 ',
 '15,600 ',
 '15,410 ',
 '4,311 ',
 '5,500',
 '14,932 ',
 '15,440 ',
 '3,130 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '4,311 ',
 '12,500 ',
 '1,192 ',
 '15,600 ',
 '2,972 ',
 '7,000 ',
 'Classifie']

In [211]:
xasil=[]
for a in xasil1:
    b=str(a).replace("~","")
    xasil.append(b)

In [212]:
xasil

['15,600 ',
 '12,050 ',
 '15,600 ',
 '15,600 ',
 '1,977 ',
 '15,600 ',
 '15,600 ',
 '12,530 ',
 '15,600 ',
 '15,410 ',
 '4,311 ',
 '5,500',
 '14,932 ',
 '15,440 ',
 '3,130 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '15,600 ',
 '4,311 ',
 '12,500 ',
 '1,192 ',
 '15,600 ',
 '2,972 ',
 '7,000 ',
 'Classifie']

In [213]:
xasil1=[]
for a in xasil:
    b=a.replace(",","")
    xasil1.append(b)

In [214]:
xasil1

['15600 ',
 '12050 ',
 '15600 ',
 '15600 ',
 '1977 ',
 '15600 ',
 '15600 ',
 '12530 ',
 '15600 ',
 '15410 ',
 '4311 ',
 '5500',
 '14932 ',
 '15440 ',
 '3130 ',
 '15600 ',
 '15600 ',
 '15600 ',
 '15600 ',
 '4311 ',
 '12500 ',
 '1192 ',
 '15600 ',
 '2972 ',
 '7000 ',
 'Classifie']

In [216]:
xasil1.remove("Classifie")

In [217]:
xasil=[]
for a in xasil1:
    b=float(a)
    xasil.append(b)

In [219]:
launch_dict['Payload mass'] = xasil

In [86]:
list_data=[]
for a in table_content_int:
    b=a.text
    list_data.append(b)

In [221]:
launch_dict['Orbit'] =table_ce[6::11]

In [223]:
launch_dict['Customer'] =table_ce[7::11]

In [224]:
launch_dict['Launchoutcome'] = table_ce[8::11]

In [231]:
Booster=[]
for a in table_ce[9::11]:
    ending=a.find("(")
    if ending==-1:
        b=a;
    else:b=a[:ending]
    Booster.append(b)

In [233]:
launch_dict['Version,Booster[b]']=Booster

In [ ]:
launch_dict['Date']=[]
launch_dict['Time']=[]

In [240]:
launch_dict['Date']=[]
for a in table_ce[1::11]:
    seperator=a.find("2020")
    b=a[:seperator+4]
    launch_dict['Date'].append(b)

In [241]:
launch_dict['Date']

['7 January 2020',
 '19 January 2020',
 '29 January 2020',
 '17 February 2020',
 '7 March 2020',
 '18 March 2020',
 '22 April 2020',
 '30 May 2020',
 '4 June 2020',
 '13 June 2020',
 '30 June 2020',
 '20 July 2020',
 '7 August 2020',
 '18 August 2020',
 '30 August 2020',
 '3 September 2020',
 '6 October 2020',
 '18 October 2020',
 '24 October 2020',
 '5 November 2020',
 '16 November 2020',
 '21 November 2020',
 '25 November 2020',
 '6 December 2020',
 '13 December 2020',
 '19 December 2020']

In [244]:
launch_dict['Time']=[]
time=[]
for a in table_ce[1::11]:
    beginning=a.find("2020")
    ending=a.find("[")
    b=a[beginning+4:ending]
    time.append(b)

In [247]:
time1=[]
for x in time:
    z=x.replace(",","")
    time1.append(z)

In [249]:
time=[]
for x in time1:
    z=x.replace(" ","")
    time.append(z)

In [251]:
launch_dict['Time']=time

In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [252]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [253]:
df

,Flight No.,Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,"Version,Booster[b]",Date,Time
0,78,"CCAFS,SLC-40",Starlink 2 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Success,7 January 2020,02:19:21
1,79,"KSC,LC-39A",Crew Dragon in-flight abort test[495](Dragon C...,12050.0,Sub-orbital[496],NASA (CTS)[497],Success,NaN,No attempt,19 January 2020,15:30
2,80,"CCAFS,SLC-40",Starlink 3 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Success,29 January 2020,14:07
3,81,"CCAFS,SLC-40",Starlink 4 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Failure,17 February 2020,15:05
4,82,"CCAFS,SLC-40",SpaceX CRS-20(Dragon C112.3 ♺),1977.0,LEO (ISS),NASA (CRS),Success,NaN,Success,7 March 2020,04:50
5,83,"KSC,LC-39A",Starlink 5 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Failure,18 March 2020,12:16
6,84,"KSC,LC-39A",Starlink 6 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Success,22 April 2020,19:30
7,85,"KSC,LC-39A",Crew Dragon Demo-2[414](Crew Dragon C206.1 End...,12530.0,LEO (ISS),NASA (CCDev),Success,NaN,Success,30 May 2020,19:22
8,86,"CCAFS,SLC-40",Starlink 7 v1.0 (60 satellites),15600.0,LEO,SpaceX,Success,NaN,Success,4 June 2020,01:25
9,87,"CCAFS,SLC-40","Starlink 8 v1.0 (58 satellites),[524][525]SkyS...",15410.0,LEO,SpaceXPlanet Labs,Success,NaN,Success,13 June 2020,09:21


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
